In [ ]:
import numpy as np
import pandas as pd
from subprocess import check_output
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# the number of initial feature
len(train.columns)

In [ ]:
#missing data in train dataset
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
import missingno as msno
msno.matrix(df=pd.concat([train,test]), color=(0.5,0,0))

In [ ]:
#dealing with missing train data
train = train.drop((missing_data[missing_data['Total'] > 1]).index,1)
train = train.drop(train.loc[train['Electrical'].isnull()].index)

In [ ]:
#filling missing test data with median
test = test.drop((missing_data[missing_data['Total'] > 1]).index,1)
#test_miss=test.isnull().sum().sort_values(ascending=False)
test=test.fillna(test.median())

In [ ]:
#df_test_miss.index[0]
len(test)

In [ ]:
#for i in df_test_miss.index:
    #test=test.drop(test[test[i].isnull()].index)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
for i in range(train.shape[1]):
    if train.iloc[:,i].dtypes == object:
        lb=LabelEncoder()
        lb.fit(list(train.iloc[:,i].values) + list(test.iloc[:,i].values))
        train.iloc[:,i]=lb.transform(list(train.iloc[:,i].values))
        test.iloc[:,i]=lb.transform(list(test.iloc[:,i].values))

In [ ]:
# keep ID for submission
train_ID = train['Id']
test_ID = test['Id']
train=train.drop('Id',axis=1)
test=test.drop('Id',axis=1)

In [ ]:
train.head()

In [ ]:
#correlation matrix
corrmat = train.corr()
f, ax = plt.subplots(figsize=(30,20))
sns.heatmap(corrmat,square=True,vmax=0.8)

In [ ]:
Y_train = train['SalePrice']
X_train = train.drop(['SalePrice'], axis=1)

In [ ]:
#import missingno as msno
msno.matrix(df=pd.concat([X_train,test]), color=(0.5,0,0))

In [ ]:
n_feature_dataset=len(X_train.columns)
n_feature_dataset

In [ ]:
# relation to the target
fig = plt.figure(figsize=(20,15))
for i in np.arange(30):
    ax = fig.add_subplot(5,6,i+1)
    sns.regplot(x=X_train.iloc[:,i], y=Y_train)

plt.tight_layout()
plt.show()

In [ ]:
ax = plt.subplots(figsize=(10,8))
sns.regplot(x=X_train['YearBuilt'], y=Y_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
regre=RandomForestRegressor(n_estimators=1000,criterion='mse',max_features=40,n_jobs=-1,oob_score=True,random_state=2)

In [ ]:
regre.fit(X_train.values,Y_train.values)

In [ ]:
Y_test_pred=regre.predict(test.values)

In [ ]:
Y_train_pred=regre.predict(X_train.values)

In [ ]:
#prediction results
Y_train_pred[:10]

In [ ]:
# raw results
Y_train.values[:10]

In [ ]:
def cv_rmsd(y,y_pred):
    y=y.values
    return np.sqrt(((y_pred-y)**2).sum()/len(y))/y.mean()

In [ ]:
cv_rmsd(Y_train,Y_train_pred)

In [ ]:
regre0=RandomForestRegressor(n_estimators=100,criterion='mae',n_jobs=-1,oob_score=True)
regre0.fit(X_train['YearBuilt'].values.reshape(-1,1),Y_train.values)

In [ ]:
cv_rmsd(Y_train,y_pred_byYearBuilt)

In [ ]:
y_pred_byYearBuilt=regre0.predict(X_train['YearBuilt'].values.reshape(-1,1))

In [ ]:
fig=plt.figure(figsize=(14,10))
sns.regplot(x=X_train['YearBuilt'], y=Y_train)

In [ ]:
fig=plt.figure(figsize=(14,10))
sns.regplot(x=X_train['YearBuilt'], y=Y_train,marker='.')
x=np.random.randint(low=1870,high=2010,size=1000).reshape(-1,1)
y=regre0.predict(x)
sns.regplot(x=x,y=y,fit_reg=False,color='r',marker='.')

In [ ]:
np.linspace(0,2018,1000).reshape(-1,1)